No nosso último papo, paramos a análise exploratória na seção de análise univariada. Nossa análise já está ganhando um corpo razoável, podemos começar a responder algumas perguntas e nos aprofundar em alguns campos específicos. Vamos começar olhando para os bairros, pois teremos muitas coisas para ver ali!

Aliás, antes de começar, todo o material que está sendo apresentado, incluindo as imagens utilizadas, estão no arquivo em anexo!

Começando pelo básico, precisamos saber quantos bairros possuem aquele dataset e quais parecem mais relevantes, no que diz respeito a quantidade de aparições (i.e., número de imóveis na localização):

In [2]:
# Bibliotecas para tratar dados
import pandas as pd
import numpy as np

# Bibliotecas para EDA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Configurar para não exibir Warnings
import warnings

warnings.filterwarnings('ignore')

# Configurar o estilo de gráfico do ggplot
plt.style.use('ggplot')

# Configurar para exibir até 15 linhas de um Dataframe do Pandas
pd.set_option ('display.max_rows', 15)

# Configurar para exibir todas as colunas de um Dataframe do Pandas
pd.set_option('display.max_columns', None)

# Configurar quebras e espaçamentos automáticas com nb_black
#%load_ext nb_black # essapoora deu errado, só desative.

aptos = pd.read_csv('Data.csv')
aptos.head()

,address,district,area,bedrooms,garage,type,rent,total
0,Rua Herval,Belenzinho,21,1,0,Studio e kitnet,2400,2939
1,Avenida São Miguel,Vila Marieta,15,1,1,Studio e kitnet,1030,1345
2,Rua Oscar Freire,Pinheiros,18,1,0,Apartamento,4000,4661
3,Rua Júlio Sayago,Vila Ré,56,2,2,Casa em condomínio,1750,1954
4,Rua Barata Ribeiro,Bela Vista,19,1,0,Studio e kitnet,4000,4654


In [3]:
print(f'Temos {aptos.district.nunique()} bairros na amostra')

Temos 1199 bairros na amostra


Bairros com maior número de imóveis?

In [5]:
top_distritos =  aptos.groupby("district").size().sort_values(ascending= False).reset_index( name='contagem')[:10]
top_distritos.columns = ['Distrito', 'Contagem']
print('Os 10 distritos com mais imóveis')
print(top_distritos)

Os 10 distritos com mais imóveis
          Distrito  Contagem
0       Bela Vista       352
1     Vila Mariana       232
2  Jardim Paulista       220
3           Centro       178
4        Pinheiros       159
5       Consolação       157
6          Santana       152
7            Mooca       147
8      Santo Amaro       146
9             Brás       141


Veja que ainda estamos com análises univariadas. Agora, que tal avançarmos? Vamos à pergunta óbvia: quais os bairros que poderiam ser taxados de "bairros de ricos"? Ou, mais formalmente, quais bairros possuem imóveis mais caros, o que significaria também que estamos falando das classes mais altas da população?

Veja no print abaixo que eu encerro o tema de cima com um comentário sobre os bairros que mais aparecem no dataset e inicio o próximo tema. A gente evita comentar absolutamente tudo, mas é interessante ir conduzindo o leitor em nosso storytelling:

E quanto ao preço de aluguel?

In [7]:
top_distritos_rent= aptos.groupby("district")['rent'].mean().sort_values(ascending= False)[:10].reset_index()
top_distritos_rent.columns = [' Distrito', 'Média de Aluguel']

top_distritos_rent['Média de Aluguel'] =  top_distritos_rent['Média de Aluguel']\
.apply(lambda x: "R {:,.2f}".format(x))

print(" Top 10 Distritos com maior média de Aluguel")
print(top_distritos_rent)

 Top 10 Distritos com maior média de Aluguel
                            Distrito Média de Aluguel
0                       Jardim Tango      R 15,000.00
1        Alphaville Residencial Dois      R 15,000.00
2        Alphaville Residencial Plus      R 15,000.00
3     Residencial Cinco (alphaville)      R 13,600.00
4          Alphaville Residencial Um      R 13,500.00
5                      Cidade Jardim      R 13,000.00
6                           Pacaembu      R 12,054.00
7                      Chácara Meyer      R 12,000.00
8         Centro Empresarial Tamboré      R 11,833.33
9  Jardim Vitoria Regia (zona Oeste)      R 11,820.00


1. Separando por partes, veja que partimos de aptos.groupby("district"), onde a função groupby() agrupa os dados pela coluna district. Ou seja, ao invés de termos as linhas para cada imóvel, temos agora um novo layout em que cada linha seria um distrito. O output, no entanto, é um objeto groupby - e não um dataframe, como estamos acostumados a ver. Este objeto pode (e costuma) ser usado para calcular estatísticas para cada grupo distinto - no nosso caso, podemos extrair as métricas por distrito.

2. Na sequência, temos ['rent'].mean(), onde estamos justamente aplicando uma estatística descritiva, a média, para cada distrito. Claro que precisamos pontuar alguma métrica para ter sua média calculada, e, no caso, pegamos o aluguel (rent). É uma sintaxe bem simples!

ATENÇÃO: Se você quiser agrupar por mais de uma categoria, lembre-se de usar colchetes para pontuar as classes que serão agrupados e duplo para as variáveis que terão a estatística calculada:

aptos.groupby(['district', 'type'])[['rent', 'total']].mean()

3. Para ordenar, do maior ao menor aluguel, usamos .sort_values(ascending=False). É importante especificar que a ordem é decrescente, para isso, usamos ascending=False.

4. O '[:10]' é apenas um fatiamento que seleciona apenas os 10 primeiros valores, ou seja, as 10 maiores médias de aluguel.

5. Por fim, resetamos o índice da Series, pois isso vai transformá-la num DataFrame. Fazemos isso com .reset_index().
6. Depois disso, apenas renomeamos as colunas, alteramos o formato da coluna "Média de Aluguel" para usar "R$" e finalizamos exibindo o resultado.

Sem grandes surpresas para quem conhece São Paulo, temos Alphaville em primeiro lugar. Entretanto, algo me incomodou aqui, que foi ter uma lista resultante com vários "Alphavilles" nela, tirando outros bairros que poderia ser interessante de ter no resultado. Este é o tipo de coisa que a gente espera que a pessoa que faça a análise questione, entende? É esperado que desperte uma curiosidade nela, questionamentos do tipo "o que será que tem além de Alphaville" ou "se ignorar Alphaville, como fica esse ranking". E foi isso que fizemos:

* **como seria o top 10 sem o Alphavile**

In [13]:
distritos_rent = aptos.groupby('district')['rent'].mean().reset_index()

distritos_rent = distritos_rent[~distritos_rent['district'].str.lower().str.contains('alphaville')]

top_distritos_rent = distritos_rent.sort_values('rent', ascending = False)[:10]

top_distritos_rent.columns = ["Distrito", "média de Aluguel"]

top_distritos_rent["Média de Aluguel"] = top_distritos_rent ["média de Aluguel"]\
.apply(lambda x: "R$ {:,.2f}".format(x))

print(" Top 10 Distritos com maior média de aluguel (sem Alphaville)")
print(top_distritos_rent)

 Top 10 Distritos com maior média de aluguel (sem Alphaville)
                              Distrito  média de Aluguel Média de Aluguel
475                       Jardim Tango      15000.000000     R$ 15,000.00
101                      Cidade Jardim      13000.000000     R$ 13,000.00
577                           Pacaembu      12054.000000     R$ 12,054.00
82                       Chácara Meyer      12000.000000     R$ 12,000.00
70          Centro Empresarial Tamboré      11833.333333     R$ 11,833.33
505  Jardim Vitoria Regia (zona Oeste)      11820.000000     R$ 11,820.00
533                 Jardim dos Estados      11583.333333     R$ 11,583.33
248                    Jardim Cordeiro      11250.000000     R$ 11,250.00
85               Chácara Santo Antônio      10650.000000     R$ 10,650.00
83                Chácara Monte Alegre      10205.000000     R$ 10,205.00


Veja que temos muito mais informação agora! Isso melhorou muito a análise e ainda deixamos claro ao gestor que estiver avaliando nosso portfólio que a gente consegue manipular os dados em Pandas sem grandes dificuldades. Como o código já é intuitivo o suficiente, com nomes adequados e sem grandes malabarismos, dispensa comentários explicando a sintaxe. Podemos adicionar ainda uma visualização, mesmo que seja redundante:

* **representando por um gráfico de barras**'

In [18]:
distritos_rent = aptos.groupby("district")["rent"].mean().reset_index()

distritos_rent =  distritos_rent[~distritos_rent['district'].str.lower().str.contains("alphaville")]

top_distritos_rent =  distritos_rent.sort_values("rent", ascending = False)[:10]

top_distritos_rent.columns = [" Distrito", "Média de Aluguel"]


